# Part 1. Compare Alabama and Tennessee orders
Background can be found at https://github.com/scottieb3/data_wrangling

The goal is to find if Bigly AL and TN orders are different.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
%matplotlib inline

## 1. Load data

In [2]:
messages = pd.read_csv('messages.csv')
orders = pd.read_csv('orders.csv')

In [3]:
orders.head()

,572987,completed_status,state,company_id,notary_id,total_notary_fee,city,appointment_date,product_name,travel_distance,docs_to_notary_sent
0,528697,completed,TN,2,32983.0,-1.0,ANTIOCH,8/19/16,refinance,8.894987,sent_by_client
1,575639,canceled,AL,1,38245.0,90.0,Montgomery,8/25/16,refinance,12.093414,sent_by_client
2,575678,completed,AL,1,64953.0,115.0,Pell City,8/25/16,refinance,19.598051,sent_by_client
3,537379,completed,TN,3,33059.0,-1.0,Johnston City,8/10/16,purchase,12.307267,sent
4,537609,completed,TN,2,32974.0,-1.0,MEMPHIS,8/3/16,refinance,5.463610,sent_by_client


In [4]:
#typo in the first column name 572987 
#fix the first column name 
orders.columns = ['order_id'] + list(orders.columns[1:])

In [5]:
#find Bigly's orders
Bigly = orders[orders.company_id == 1]
Bigly.state.value_counts()

TN    165
AL    132
Name: state, dtype: int64

In [6]:
Bigly.isnull().sum()

order_id                0
completed_status        0
state                   0
company_id              0
notary_id              21
total_notary_fee        0
city                    0
appointment_date        0
product_name            0
travel_distance        16
docs_to_notary_sent    32
dtype: int64

In [7]:
AL = Bigly[Bigly.state == 'AL']
TN = Bigly[Bigly.state == 'TN']

## 2. Order statistics
To compare AL and TN ordes, examine test statistics for each paramter in the orders table. The assumption is that the two orders are independent. All tests are evaluated at 95% confidence level.
1. t-test : notary travel distance, total notary fee;
2. chi-squared test : completed status, product category, appointment date.

### 1). Notary travel distance
At 95% confidence level, there is no difference in the notary travel distance between the two orders. 
1. Calcualte the mean and standard deviation just to get a sense of the difference.
2. f-test to compare variance. The obtained p value is larger than the significance level (0.05), so fail to reject the null hypothesis of equal variance.
3. t-test to compare mean. The p value is larger than the significance level (0.05), so fail to reject the null hypothesis. 

In [8]:
#mean and std of distance
Distance_AL = AL.travel_distance.dropna().as_matrix()
Distance_TN = TN.travel_distance.dropna().as_matrix()
print "AL: mean = %f, std = %f"%(Distance_AL.mean(), Distance_AL.std())
print "TN: mean = %f and std = %f"%(Distance_TN.mean(), Distance_TN.std())

AL: mean = 15.361803, std = 11.835865
TN: mean = 14.490850 and std = 11.919738


In [9]:
#f-test to compare variance 
X = Distance_AL
Y = Distance_TN
F = np.var(X) / np.var(Y)
df1 = len(X) - 1 #degree of freedom
df2 = len(Y) - 1
stats.f.cdf(F, df1, df2)

0.46955175163335022

In [10]:
#t-test to compare mean
stats.ttest_ind(a=X, b=Y, equal_var=True)

Ttest_indResult(statistic=0.60841517897863051, pvalue=0.54340714118440947)

### 2). Total Notary Fee
At 95% confidence level, the total notary fee between AL and TN are different.

Follow the previous analysis of travel distance. First compare variance using f-test, and then compare mean using t-test. The p values in both tests are smaller than the significance level, so reject null hypothesis. 

In [11]:
#mean and std
Fee_AL = AL.total_notary_fee.dropna().as_matrix()
Fee_TN = TN.total_notary_fee.dropna().as_matrix()
print "AL: mean = %f, std = %f"%(Fee_AL.mean(), Fee_AL.std())
print "TN: mean = %f and std = %f"%(Fee_TN.mean(), Fee_TN.std())

AL: mean = 89.750000, std = 20.130399
TN: mean = 84.175758 and std = 23.466792


In [12]:
#f-test to compare variance
X = Fee_AL
Y = Fee_TN
F = np.var(X) / np.var(Y)
df1 = len(X) - 1
df2 = len(Y) - 1
stats.f.cdf(F, df1, df2)

0.033812868491198973

In [13]:
#t-test to compare mean
stats.ttest_ind(a=X, b=Y, equal_var=False)

Ttest_indResult(statistic=2.1946433075089229, pvalue=0.028972498024962757)

### 3). Product category
At 95% confidence level, the product category between the two orders are different. 

Chi-squared test to compare discrete samples. 

In [14]:
print AL.product_name.value_counts()
sum_AL = sum(AL.product_name.value_counts())
print sum_AL

refinance    115
purchase      17
Name: product_name, dtype: int64
132


In [15]:
print TN.product_name.value_counts()
sum_TN = sum(TN.product_name.value_counts())
print sum_TN

refinance           100
purchase             63
reverse mortgage      2
Name: product_name, dtype: int64
165


In [16]:
#chi-squared test
stats.chisquare(f_obs = np.array([115, 17, 0]), f_exp = np.array([100,63,2])*sum_AL/sum_TN)

Power_divergenceResult(statistic=38.092500000000001, pvalue=5.3495681442367345e-09)

### 4). Completed Status
At 95% confidence level, the completed status between the two states are different. AL tends to have more canceled orders.

Chi-squared test to compare discrete samples. 

In [17]:
print AL.completed_status.value_counts()
print 'Alabama has a signing rate of "{0:.0f}%"'.format(AL[AL.completed_status=='completed'].shape[0]/float(sum_AL)*100)

completed       78
canceled        51
did_not_sign     3
Name: completed_status, dtype: int64
Alabama has a signing rate of "59%"


In [18]:
print TN.completed_status.value_counts()
print 'Tennessee has a signing rate of "{0:.0f}%"'.format(TN[TN.completed_status=='completed'].shape[0]/float(sum_TN)*100)

completed       149
canceled         14
did_not_sign      2
Name: completed_status, dtype: int64
Tennessee has a signing rate of "90%"


In [19]:
#chi-squared test 
stats.chisquare(f_obs = np.array([78,51,3]), f_exp = np.array([149,14,2])*sum_AL/sum_TN)

Power_divergenceResult(statistic=163.58059587471354, pvalue=3.0124852737674572e-36)

### 5). How the docs are sent to notary
At 95% confidence level, the way that the docs are sent to notary are different between the two orders.

Chi-squared test to compare the way docs are sent to notary.

In [20]:
print AL.docs_to_notary_sent.value_counts()
sum_sent_AL = sum(AL.docs_to_notary_sent.value_counts())
print sum_sent_AL

sent                 60
sent_by_client       46
emailed_to_notary     7
direct_links          2
overnighted           1
Name: docs_to_notary_sent, dtype: int64
116


In [21]:
print TN.docs_to_notary_sent.value_counts()
sum_sent_TN = sum(TN.docs_to_notary_sent.value_counts())
print sum_sent_TN

sent                 83
sent_by_client       53
direct_links          8
emailed_to_notary     4
at_closing            1
Name: docs_to_notary_sent, dtype: int64
149


In [22]:
#chi-squared test 
stats.chisquare(f_obs = np.array([60,46,7,2]), f_exp = np.array([83,53,4,8])*sum_sent_AL/sum_sent_TN)

Power_divergenceResult(statistic=8.8597560975609753, pvalue=0.031214803695609718)

### 6). Appointment Date
At 95% confidence level, there is no difference of appointment dates between the two states.

Appointment date are included for the completeness of the study. Mainly look at the year. Both states have all orderes in year 2016. 

In [23]:
AL.appointment_date.apply(lambda x:x[-2:]).value_counts()

16    132
Name: appointment_date, dtype: int64

In [24]:
TN.appointment_date.apply(lambda x:x[-2:]).value_counts()

16    165
Name: appointment_date, dtype: int64

## 3. Conclusion
At 95% confidence level, the Bigly orders in Alabama and Tennessee are different, including the total notary fee, product catogory, completed status, and the way docs are sent to notary. 

In particular, the signing rate is lower in Alabama of < 60%, while the rate is > 90% in Tennessee. In addition, the two orders handle different product profiles, and they differ in the way that docs are sent to notaries. This implies that the two Bigly schedule teams may handle orders differently. 

The notary travel distance and appointment date seem consistent between the two orders.